In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
projectbasepath = "../../"
rawdatapath = os.path.join(projectbasepath, "rawdata/texas/")
outdatapath = os.path.join(projectbasepath, "data/local/texas/")

In [3]:
sheets1 = pd.read_excel(os.path.join(rawdatapath, "hospitalizations.xlsx"), sheet_name=None, header=2)
sheets2 = pd.read_excel(os.path.join(rawdatapath, "capacity.xlsx"), sheet_name=None, header=2)
sheets = dict(sheets1, **sheets2)

In [4]:
metric_name_cvt = {
    "COVID-19 Hospitalizations": "patients_covid_total",
    "COVID-19 General Beds": "patients_covid_ward",
    "COVID-19 ICU": "patients_covid_icu",
    "Total Available Beds": "beds_avail_total",
    "ICU Beds Available": "beds_avail_icu",
    "Total Occupied Beds": "patients_all_total",
    "ICU Beds Occupied": "patients_all_icu",
}

In [5]:
rawdata = {}
tsaids = [chr(i) for i in range(ord('A'), ord('V')+1)]
for k,v in sheets.items():
    v = v.rename(columns={"TSA ID": "tsa_id", "TSA AREA": "tsa_name"})
    v.loc[:,"tsa_id"] = [x[0] if (isinstance(x, str) and len(x)==2) else x for x in v["tsa_id"].values]
    m = metric_name_cvt[k]
    rawdata[m] = v[v["tsa_id"].isin(tsaids)]

In [6]:
rawdata["patients_covid_total"]

,tsa_id,tsa_name,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,...,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19
0,A,Amarillo,17.0,17.0,17.0,40.0,18.0,24.0,18.0,35.0,...,47.0,44.0,42.0,46.0,68.0,72.0,70.0,71.0,62.0,63.0
1,B,Lubbock,34.0,35.0,31.0,34.0,32.0,38.0,35.0,32.0,...,85.0,81.0,83.0,84.0,90.0,86.0,84.0,75.0,74.0,90.0
2,C,Wichita Falls,5.0,5.0,6.0,4.0,5.0,0.0,0.0,0.0,...,34.0,29.0,31.0,28.0,27.0,31.0,31.0,36.0,39.0,29.0
3,D,Abilene,6.0,7.0,2.0,6.0,5.0,8.0,3.0,10.0,...,30.0,27.0,27.0,29.0,30.0,29.0,31.0,32.0,31.0,28.0
4,E,Dallas/Ft. Worth,369.0,365.0,373.0,449.0,470.0,476.0,428.0,435.0,...,780.0,750.0,741.0,762.0,776.0,742.0,732.0,754.0,754.0,733.0
5,F,Paris,14.0,16.0,21.0,18.0,15.0,12.0,12.0,17.0,...,81.0,82.0,70.0,74.0,78.0,69.0,69.0,67.0,64.0,67.0
6,G,Longview/Tyler,32.0,26.0,33.0,32.0,36.0,43.0,36.0,42.0,...,174.0,188.0,172.0,172.0,168.0,169.0,170.0,177.0,164.0,159.0
7,H,Lufkin,15.0,18.0,23.0,15.0,28.0,24.0,37.0,27.0,...,44.0,43.0,39.0,24.0,31.0,41.0,44.0,48.0,42.0,31.0
8,I,El Paso,14.0,23.0,12.0,30.0,24.0,28.0,33.0,24.0,...,110.0,113.0,114.0,110.0,110.0,114.0,118.0,122.0,128.0,138.0
9,J,Midland/Odessa,10.0,11.0,17.0,7.0,10.0,15.0,12.0,10.0,...,67.0,71.0,65.0,64.0,55.0,62.0,63.0,73.0,56.0,58.0


In [7]:
date_cvt = {
    "39668": "2020-08-08",
    "44050": "2020-08-07",
    "44051": "2020-08-08",
    "44059": "2020-08-16",
    "2020-08-16.y": "2020-08-16",
}

In [8]:
data_rows = []
for k,df in rawdata.items():
    for index, row in df.iterrows():
        for _col in df.columns[2:]:
            col = _col if _col not in date_cvt else date_cvt[_col]
            val = row[_col]
            val = np.nan if val == "--" else val
            val = float(val) if isinstance(val, str) else val
            val = int(val) if (isinstance(val, float) and not np.isnan(val)) else val
            data_rows.append({"tsa_id": row.tsa_id, "tsa_name": row.tsa_name, "date": col, k: val})

In [9]:
_df = pd.DataFrame(data_rows)
data = _df.groupby(by=["tsa_id", "tsa_name", "date"]).agg(lambda x: np.nan if np.isnan(x).all() else np.sum(x)).reset_index()
data

,tsa_id,tsa_name,date,patients_covid_total,patients_covid_ward,patients_covid_icu,beds_avail_total,beds_avail_icu,patients_all_total,patients_all_icu
0,A,Amarillo,2020-04-12,17.0,13.0,4.0,613.0,77.0,417.0,NaN
1,A,Amarillo,2020-04-13,17.0,12.0,5.0,251.0,78.0,427.0,NaN
2,A,Amarillo,2020-04-14,17.0,12.0,5.0,571.0,67.0,461.0,NaN
3,A,Amarillo,2020-04-15,40.0,38.0,2.0,519.0,54.0,322.0,NaN
4,A,Amarillo,2020-04-16,18.0,12.0,6.0,668.0,92.0,522.0,NaN
...,...,...,...,...,...,...,...,...,...,...
3537,V,Lower Rio Grande Valley,2020-09-15,360.0,218.0,141.0,993.0,130.0,1763.0,343.0
3538,V,Lower Rio Grande Valley,2020-09-16,346.0,210.0,133.0,1011.0,139.0,1846.0,339.0
3539,V,Lower Rio Grande Valley,2020-09-17,347.0,211.0,133.0,1008.0,148.0,1938.0,332.0
3540,V,Lower Rio Grande Valley,2020-09-18,341.0,204.0,134.0,983.0,140.0,1993.0,329.0


In [10]:
data.to_csv(os.path.join(outdatapath, "tx_tsa_hospitalizations.csv"), index=False)